In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import os

#defog=91, tdcsfog=833

seriesWithEvents = []
with open('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/events.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if row[0] != 'Id' and row[0] not in seriesWithEvents:
            seriesWithEvents.append(row[0])

#print(seriesWithEvents)
print('Number of series with events: ', len(seriesWithEvents))

defogCt, tdcsfogCt, notypeCt = 0, 0, 0
defogSeries, tdcsfogSeries, notypeSeries = [], [], []
for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'):
    for filename in filenames:
        if 'unlabeled' not in dirname and 'test' not in dirname and filename not in ['sample_submission.csv','subjects.csv','tasks.csv','defog_metadata.csv','daily_metadata.csv','events.csv','tdcsfog_metadata.csv'] and filename.replace('.csv', '') in seriesWithEvents:
            if 'defog' in dirname: 
                defogCt += 1
                defogSeries.append(filename)
            if 'tdcsfog' in dirname: 
                tdcsfogCt += 1
                tdcsfogSeries.append(filename)
            if 'notype' in dirname: 
                notypeCt += 1
                notypeSeries.append(filename)
            #print(os.path.join(dirname.replace('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/', ''), filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print('defogCt: ', defogCt, ', tdcsfogCt: ',tdcsfogCt, ', notypeCt: ', notypeCt)

Number of series with events:  535
defogCt:  91 , tdcsfogCt:  411 , notypeCt:  33


In [2]:
from decimal import Decimal
import shutil

trainPath = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/'
defogBasePath, tdcsfogBasePath, notypeBasePath = trainPath+'defog/',trainPath+'tdcsfog/', trainPath+'notype/'

In [3]:
def calculateJerkWinAvSq(Acc_2_prev, Acc_1_prev, Acc_0, Acc_1, Acc_2, sampleRate):
    timeStep = Decimal(1/sampleRate)
    return(Decimal(0.125)*(((Acc_1_prev-Acc_2_prev)/timeStep)**2+((Acc_2-Acc_1)/timeStep)**2)+Decimal(0.375)*(((Acc_0-Acc_1_prev)/timeStep)**2+((Acc_1-Acc_0)/timeStep)**2)) 

def calculateJWASEdge(Acc_0, Acc_adjacent, Acc_next_adjacent, sampleRate):
    timeStep = Decimal(1/sampleRate)
    return(Decimal(0.75)*(((Acc_adjacent - Acc_0)/timeStep)**2)+Decimal(0.25)*(((Acc_next_adjacent - Acc_adjacent)/timeStep)**2))

def calculateJWASEdgeAdjacent(Acc_0, Acc_adjacent_1, Acc_adjacent_2, Acc_next_adjacent, sampleRate):
    timeStep = Decimal(1/sampleRate)
    return(Decimal(0.375)*(((Acc_0 - Acc_adjacent_1)**2+((Acc_0 - Acc_adjacent_2)/timeStep)**2))+Decimal(0.25)*(((Acc_next_adjacent - Acc_adjacent_2)/timeStep)**2))

def initOutputDirectory(outputPath):
    try:
        os.mkdir(outputPath)
    except:
        shutil.rmtree(outputPath)
        os.mkdir(outputPath)

def preProcessDefog(basePath, fileList):
    outputPath = '/kaggle/working/pruned_'+basePath.replace(trainPath, '')
    initOutputDirectory(outputPath)
    
    AccVIndex, AccMLIndex, AccAPIndex = 1, 2, 3
    StartHesIndex, WalkIndex = 4, 6
    
    for file in fileList:
        with open(basePath+file, 'r', newline='') as ogCsv, open(outputPath+file, 'w', newline='') as outputCsv:
            ogCsvReader = csv.reader(ogCsv, delimiter=',', quotechar='|')
            outputCsvWriter = csv.writer(outputCsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            listOgCsvReader = list(ogCsvReader)
            numRows = len(listOgCsvReader)
            for rowNum, row in enumerate(listOgCsvReader):
                if rowNum == 0 or (row[-1] == 'Task' and row[-2] == 'Valid'):
                    outputCsvWriter.writerow(row + ['JerkWinAvSqV', 'JerkWinAvSqML', 'JerkWinAvSqAP'])
                elif rowNum == 1:
                    JerkWinAvSqV = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), 128)
                    JerkWinAvSqML = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), 128)
                    JerkWinAvSqAP = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), 128)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == 2:
                    JerkWinAvSqV = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), 128)
                    JerkWinAvSqML = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), 128)
                    JerkWinAvSqAP = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), 128)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == numRows-1:
                    JerkWinAvSqV = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum-2][AccVIndex]), 128)
                    JerkWinAvSqML = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), 128)
                    JerkWinAvSqAP = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), 128)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == numRows-2:
                    JerkWinAvSqV = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum-2][AccVIndex]), 128)
                    JerkWinAvSqML = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), 128)
                    JerkWinAvSqAP = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), 128)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif (row[-1] == 'true' and row[-2] == 'true'):
                    JerkWinAvSqV = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), 128)
                    JerkWinAvSqML = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), 128)
                    JerkWinAvSqAP = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), 128)
                    if int(row[StartHesIndex]) == 1 or int(row[WalkIndex]) == 1:
                        for x in range(4):
                            delta = Decimal((1+x)/100)
                            outputCsvWriter.writerow(row + [(JerkWinAvSqV*delta), (JerkWinAvSqML*delta), (JerkWinAvSqAP*delta)])
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])


In [4]:
preProcessDefog(defogBasePath, defogSeries)
#preProcessDefog(notypeBasePath, notypeSeries)

In [5]:
# read output defog file 0
# with open('/kaggle/working/pruned_'+defogBasePath.replace(trainPath, '')+defogSeries[0]) as fd:
#     reader=csv.reader(fd)
#     for row in reader:
#         print(row)

In [6]:
# https://www.hindawi.com/journals/bmri/2018/7816160/
def preProcessTdcsfog(basePath, fileList, outputPath='', sampleRate=100):
    isTrainingData = False
    
    if outputPath == '':
        isTrainingData = True
        outputPath = '/kaggle/working/pruned_'+basePath.replace(trainPath, '')
        initOutputDirectory(outputPath)
    
    AccVIndex, AccMLIndex, AccAPIndex = 1, 2, 3
    StartHesIndex, WalkIndex = 4, 6
    
    for file in fileList:
        with open(basePath+file, 'r', newline='') as ogCsv, open(outputPath+file, 'w', newline='') as outputCsv:
            ogCsvReader = csv.reader(ogCsv, delimiter=',', quotechar='|')
            outputCsvWriter = csv.writer(outputCsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
            listOgCsvReader = list(ogCsvReader)
            numRows = len(listOgCsvReader)
            for rowNum, row in enumerate(listOgCsvReader):
                if rowNum == 0:
                    outputCsvWriter.writerow(row + ['JerkWinAvSqV', 'JerkWinAvSqML', 'JerkWinAvSqAP'])
                elif rowNum == 1:
                    JerkWinAvSqV = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), sampleRate)
                    JerkWinAvSqML = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), sampleRate)
                    JerkWinAvSqAP = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), sampleRate)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == 2:
                    JerkWinAvSqV = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), sampleRate)
                    JerkWinAvSqML = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), sampleRate)
                    JerkWinAvSqAP = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), sampleRate)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == numRows-1:
                    JerkWinAvSqV = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum-2][AccVIndex]), sampleRate)
                    JerkWinAvSqML = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), sampleRate)
                    JerkWinAvSqAP = calculateJWASEdge(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), sampleRate)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                elif rowNum == numRows-2:
                    JerkWinAvSqV = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum-2][AccVIndex]), sampleRate)
                    JerkWinAvSqML = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), sampleRate)
                    JerkWinAvSqAP = calculateJWASEdgeAdjacent(Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), sampleRate)
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])
                else:
                    JerkWinAvSqV = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccVIndex]), Decimal(listOgCsvReader[rowNum-1][AccVIndex]), Decimal(listOgCsvReader[rowNum][AccVIndex]), Decimal(listOgCsvReader[rowNum+1][AccVIndex]), Decimal(listOgCsvReader[rowNum+2][AccVIndex]), sampleRate)
                    JerkWinAvSqML = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccMLIndex]), Decimal(listOgCsvReader[rowNum-1][AccMLIndex]), Decimal(listOgCsvReader[rowNum][AccMLIndex]), Decimal(listOgCsvReader[rowNum+1][AccMLIndex]), Decimal(listOgCsvReader[rowNum+2][AccMLIndex]), sampleRate)
                    JerkWinAvSqAP = calculateJerkWinAvSq(Decimal(listOgCsvReader[rowNum-2][AccAPIndex]), Decimal(listOgCsvReader[rowNum-1][AccAPIndex]), Decimal(listOgCsvReader[rowNum][AccAPIndex]), Decimal(listOgCsvReader[rowNum+1][AccAPIndex]), Decimal(listOgCsvReader[rowNum+2][AccAPIndex]), sampleRate)
                    if isTrainingData and (int(row[StartHesIndex]) == 1 or int(row[WalkIndex]) == 1):
                        for x in range(2):
                            delta = Decimal((1+x)/100)
                            outputCsvWriter.writerow(row + [(JerkWinAvSqV*delta), (JerkWinAvSqML*delta), (JerkWinAvSqAP*delta)])
                    outputCsvWriter.writerow(row + [JerkWinAvSqV, JerkWinAvSqML, JerkWinAvSqAP])


In [7]:
preProcessTdcsfog(tdcsfogBasePath, tdcsfogSeries)

In [8]:
# a = 0
# for dirname, _, filenames in os.walk('/kaggle/working/pruned_tdcsfog/'):
#     for filename in filenames:
#         print(filename)
#         a+=1
# print(a)

In [9]:
!pip install keras-tcn --no-index --find-links=file:///kaggle/input/keras-tcn/ 

Looking in links: file:///kaggle/input/keras-tcn/
Processing /kaggle/input/keras-tcn/keras_tcn-3.5.0-py3-none-any.whl


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
try:
    from tcn import TCN
except: 
    import sys
    !{sys.executable} -m pip install keras-tcn
    from tcn import TCN
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
def createJWASmodel(basePath1, basePath2, fileList1, fileList2):
    # Load and concatenate multiple CSV files
    dataframes1 = [pd.read_csv('/kaggle/working/pruned_'+basePath1.replace(trainPath, '')+file) for file in fileList1]
    data1 = pd.concat(dataframes1)

    X1 = data1[['JerkWinAvSqV', 'JerkWinAvSqML', 'JerkWinAvSqAP']]
    y1 = data1[['StartHesitation', 'Turn', 'Walking']]
    
    dataframes2 = [pd.read_csv('/kaggle/working/pruned_'+basePath2.replace(trainPath, '')+file) for file in fileList2]
    data2 = pd.concat(dataframes2)

    X2 = data2[['JerkWinAvSqV', 'JerkWinAvSqML', 'JerkWinAvSqAP']]
    y2 = data2[['StartHesitation', 'Turn', 'Walking']]

    X = pd.concat([X1, X2], axis=0)
    y = pd.concat([y1, y2], axis=0)
    
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the input features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Reshape the input data for compatibility with TCN
    X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    def buildModel(hyperParams):
        model = Sequential()

        model.add(TCN(
            hyperParams.Int('units', min_value=32, max_value=256, step=32, default=32),
            input_shape=(X_train_reshaped.shape[1], 1)
        ))
        model.add(Dropout(
            hyperParams.Float('dropout', 0, 0.5, step=0.1, default=0.5)
        ))
        model.add(Dense(3, activation='sigmoid'))

        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    tuner = RandomSearch(
    buildModel,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='tuner_results',
    project_name='tuning_tcn'
    )
    
    tuner.search_space_summary()
    tuner.search(X_train_reshaped, y_train, epochs=5, validation_split=0.2, batch_size=512)
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build and train the TCN model
    model = buildModel(best_hyperparameters)
    model.fit(X_train_reshaped, y_train, epochs=5, batch_size=512, validation_split=0.2)

    # Predict on test data and evaluate the performance
    y_pred = np.round(model.predict(X_test_reshaped))
    print(classification_report(y_test, y_pred, target_names=['StartHesitation', 'Turn', 'Walking']))
    return model, scaler

In [12]:
# tdcsfogModel, tdcsfogScaler = createJWASmodel(tdcsfogBasePath, tdcsfogSeries)
# defogModel, defogScaler = createJWASmodel(defogBasePath, defogSeries)

fogModel, fogScaler = createJWASmodel(tdcsfogBasePath, defogBasePath, tdcsfogSeries, defogSeries)

Trial 5 Complete [00h 36m 17s]
val_accuracy: 0.24755113571882248

Best val_accuracy So Far: 0.2573413699865341
Total elapsed time: 04h 34m 17s
Epoch 1/5
12692/12692 [==============================] - 323s 25ms/step - loss: 0.3809 - accuracy: 0.2328 - val_loss: 0.3792 - val_accuracy: 0.2334
Epoch 2/5
12692/12692 [==============================] - 336s 26ms/step - loss: 0.3781 - accuracy: 0.2409 - val_loss: 0.3762 - val_accuracy: 0.2428
Epoch 3/5
12692/12692 [==============================] - 334s 26ms/step - loss: 0.3775 - accuracy: 0.2438 - val_loss: 0.3777 - val_accuracy: 0.2533
Epoch 4/5
12692/12692 [==============================] - 334s 26ms/step - loss: 0.3773 - accuracy: 0.2459 - val_loss: 0.3762 - val_accuracy: 0.2469
Epoch 5/5
63459/63459 [==============================] - 187s 3ms/step


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       0.00      0.00      0.00    184130
           Turn       0.55      0.06      0.10    453586
        Walking       0.00      0.00      0.00    223217

      micro avg       0.55      0.03      0.06    860933
      macro avg       0.18      0.02      0.03    860933
   weighted avg       0.29      0.03      0.05    860933
    samples avg       0.01      0.01      0.01    860933



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
testPath = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/'
testFileDict = {'defog/02ab235146.csv': 128,'tdcsfog/003f117e14.csv': 100}

def processTestingData(testPath, testFileDict):
    for fileName, sampleRate in testFileDict.items():
        preProcessTdcsfog(testPath, [fileName], '/kaggle/working/processed_test/', sampleRate)

def generateFinalPredictions(fileName, finalModel, scaler, outputWriteMode = 'a'):
        test_data = pd.read_csv('/kaggle/working/processed_test/'+fileName)

        X_test_final = test_data[['JerkWinAvSqV', 'JerkWinAvSqML', 'JerkWinAvSqAP']]

        # Scale the input features using the same scaler fit on the training data
        X_test_final = scaler.transform(X_test_final)

        # Reshape the input data for compatibility with TCN
        X_test_final_reshaped = X_test_final.reshape(X_test_final.shape[0], X_test_final.shape[1], 1)

        # Predict
        y_test_final_pred = np.round(finalModel.predict(X_test_final_reshaped))

        # The y_test_final_pred contains the predicted labels for the test files
        print(y_test_final_pred)
        
        patientId = fileName.split('/')[-1].replace('.csv', '')
        submissions = pd.DataFrame()
        submissions['Id'] = [patientId+'_'+str(x) for x in range (len(y_test_final_pred))]
        submissions['StartHesitation'] = y_test_final_pred[:, 0]
        submissions['Turn'] = y_test_final_pred[:, 1]
        submissions['Walking'] = y_test_final_pred[:, 2]
        submissions.to_csv('/kaggle/working/submission.csv', index=False, header=(outputWriteMode == 'w'), mode=outputWriteMode)
        
        return(y_test_final_pred)
    

In [14]:
initOutputDirectory('/kaggle/working/processed_test')
initOutputDirectory('/kaggle/working/processed_test/defog')
initOutputDirectory('/kaggle/working/processed_test/tdcsfog')
processTestingData(testPath, testFileDict)

In [15]:
initOutputDirectory('/kaggle/working/output')

In [16]:
finalDefogPred = generateFinalPredictions('defog/02ab235146.csv', fogModel, fogScaler,outputWriteMode = 'w')

8803/8803 [==============================] - 26s 3ms/step
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [17]:
finalTdcsfogPred = generateFinalPredictions('tdcsfog/003f117e14.csv', fogModel, fogScaler)

147/147 [==============================] - 1s 4ms/step
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [18]:
shutil.rmtree('/kaggle/working/processed_test')
shutil.rmtree('/kaggle/working/pruned_defog')
shutil.rmtree('/kaggle/working/pruned_tdcsfog')
shutil.rmtree('/kaggle/working/tuner_results')